In [1]:
import cv2
import numpy as np

In [2]:
def preprocess(img):
    height = img.shape[0]
    width = img.shape[1]
    roi_vertices =[
        (0, height),
        (width*0.5, height*0.6),
        (width, height*0.8)
    ]
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    canny = cv2.Canny(gray,0,255)
    cropped = roi(canny,np.array([roi_vertices],np.int32))
    cv2.imshow('c',cropped)
    return cropped
def roi(img,vertices):
    mask = np.zeros_like(img)
    cv2.fillPoly(mask,vertices,255)
    masked_image=cv2.bitwise_and(img,mask)
    cv2.imshow('a',masked_image)
    return masked_image
def draw_hough_lines(img,lines):
    img = np.copy(img)
    blank_image = np.zeros((img.shape[0],img.shape[1],3),dtype=np.uint8)
    try:
        for line in lines:
            for x1,y1,x2,y2 in line:
                cv2.line(blank_image,(x1,y1),(x2,y2),(255,0,255),thickness=5)
    except:
        pass
    cv2.imshow('l',blank_image)
    img = cv2.addWeighted(img,1,blank_image,1,0.0)
    return img

In [15]:
def apply_code(path,file_saved):
    vid = cv2.VideoCapture(path)
    frame_width=int(vid.get(3))
    frame_height=int(vid.get(4))
    size = (frame_width, frame_height)
    result=cv2.VideoWriter(file_saved,cv2.VideoWriter_fourcc(*'MJPG'),10, size)
    while True:
        ret,frame = vid.read()
        if frame is None:
            break
        cropped= preprocess(frame)
        lines = cv2.HoughLinesP(cropped,rho=8,threshold=255,theta=np.pi/180,minLineLength=60,maxLineGap=150)
        img = draw_hough_lines(frame,lines)
        cv2.imshow('Lane detection',img)
        result.write(img)
        if cv2.waitKey(1) & 0xFF == ord('x'):
            break
    result.release()
    vid.release()
    cv2.destroyAllWindows()

In [16]:
apply_code('project_video.mp4','project_out_video.avi')